# CSE314 Final Project

In [2]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import scipy as sp
from glob import glob
import re
import json
import tqdm
import ast
import dash
import os
import sys


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


ModuleNotFoundError: No module named 'scipy'

In [3]:
sys.path.append('/home/themaster/workspace/cse314final/MovieClean')
from MovieClean import norm

In [4]:
path = "/media/work_data/DS_final_project"
file_names = !ls $path
file_names

['ls: /media/work_data/DS_final_project: No such file or directory']

In [5]:
save_path = './cse314final/'

# Clean Credits

In [6]:
credits = pd.read_csv(path+"/"+file_names[1])
credits

IndexError: list index out of range

In [6]:
all_cast = norm.json2df(credits, 'cast', 'id')
all_cast

,cast_id,character,credit_id,gender,id,name,order,profile_path
0,14.0,Woody (voice),52fe4284c3a36847f8024f95,2.0,862.0,Tom Hanks,0.0,/pQFoyx7rp09CJTAb932F2g8Nlho.jpg
1,15.0,Buzz Lightyear (voice),52fe4284c3a36847f8024f99,2.0,862.0,Tim Allen,1.0,/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg
2,16.0,Mr. Potato Head (voice),52fe4284c3a36847f8024f9d,2.0,862.0,Don Rickles,2.0,/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg
3,17.0,Slinky Dog (voice),52fe4284c3a36847f8024fa1,2.0,862.0,Jim Varney,3.0,/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg
4,18.0,Rex (voice),52fe4284c3a36847f8024fa5,2.0,862.0,Wallace Shawn,4.0,/oGE6JqPP2xH4tNORKNqxbNPYi7u.jpg
...,...,...,...,...,...,...,...,...
0,2.0,,52fe4ea59251416c7515d7d5,2.0,227506.0,Iwan Mosschuchin,0.0,None
1,3.0,,52fe4ea59251416c7515d7d9,1.0,227506.0,Nathalie Lissenko,1.0,None
2,4.0,,52fe4ea59251416c7515d7dd,2.0,227506.0,Pavel Pavlov,2.0,None
3,5.0,,52fe4ea59251416c7515d7e1,0.0,227506.0,Aleksandr Chabrov,3.0,None


In [7]:
actors = norm.split_table(all_cast, 'name', ['cast_id', 'character', 'credit_id', 'gender', 'order', 'profile_path'])
actors.drop(columns=['credit_id', 'order', 'profile_path', 'cast_id'], inplace=True)
actors

,character,gender
name,,
Tom Hanks,Woody (voice),2.0
Tim Allen,Buzz Lightyear (voice),2.0
Don Rickles,Mr. Potato Head (voice),2.0
Jim Varney,Slinky Dog (voice),2.0
Wallace Shawn,Rex (voice),2.0
...,...,...
Iwan Mosschuchin,,2.0
Nathalie Lissenko,,1.0
Pavel Pavlov,,2.0


In [60]:
movies_names = all_cast.drop(columns=['cast_id', 'character', 'credit_id', 'gender', 'order', 'profile_path'])
movies_names = movies_names.astype({'id':int})
movies_names = movies_names.set_index('id')
movies_names.index

Int64Index([   862,    862,    862,    862,    862,    862,    862,    862,
               862,    862,
            ...
             67758,  67758,  67758,  67758,  67758, 227506, 227506, 227506,
            227506, 227506],
           dtype='int64', name='id', length=562474)

In [9]:
#Checkpoint
movies_names.to_csv(save_path+'movies_names.csv', encoding='utf-8')
actors.to_csv(save_path+'actors.csv', encoding='utf-8')

# Clean Keywords

In [11]:
keywords = pd.read_csv(path+"/"+file_names[2])
keywords.dtypes

id           int64
keywords    object
dtype: object

In [13]:
keywords = norm.json2series(keywords, 'keywords', 'name', 'id').rename('keyword')
keywords.dtypes

AttributeError: 'Series' object has no attribute 'set_index'

In [87]:
#Checkpoint
keywords.to_csv('./cse314final/keywords.csv', encoding='utf-8')

# Clean Movies Metadata

In [14]:
movies = pd.read_csv(path+"/"+file_names[5])
movies.head()

/tmp/ipykernel_45060/663887663.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies = pd.read_csv(path+"/"+file_names[5])


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [49]:
collections = (
    norm.json2series(movies[['belongs_to_collection', 'id']]
                  .dropna(), 'belongs_to_collection', 'name', 'id')
    .rename('collection')
)
collections

id
862                 Toy Story Collection
15602          Grumpy Old Men Collection
11862     Father of the Bride Collection
710                James Bond Collection
21032                   Balto Collection
                       ...              
24568            The Carry On Collection
19307            The Carry On Collection
21251            The Carry On Collection
460135    DC Super Hero Girls Collection
182030              Red Lotus Collection
Name: collection, Length: 4494, dtype: object

In [58]:
collections = norm.id2int(collections)

In [52]:
genres = norm.json2series(movies[['genres', 'id']].dropna(), 'genres', 'name', 'id').rename('genre')
genres = norm.id2int(genres)
genres

/home/themaster/workspace/cse314final/MovieClean/MovieClean/norm.py:33: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  .apply(pd.Series)


,genre
id,
862,Animation
8844,Adventure
15602,Romance
31357,Comedy
11862,Comedy
...,...
220447,Drama
90063,Animation
11052,Family


In [53]:
languages = norm.json2series(movies[['id', 'spoken_languages']].dropna(), 'spoken_languages', 'name', 'id').rename('language')
languages = norm.id2int(languages)
languages

/home/themaster/workspace/cse314final/MovieClean/MovieClean/norm.py:33: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  .apply(pd.Series)


,language
id,
862,English
8844,English
15602,English
31357,English
11862,English
...,...
8985,Español
8985,svenska
8985,Türkçe


In [54]:
companies = norm.json2series(movies[['id','production_companies']].dropna(), 'production_companies', 'name', 'id').rename('company')
companies = norm.id2int(companies)
companies

/home/themaster/workspace/cse314final/MovieClean/MovieClean/norm.py:33: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  .apply(pd.Series)
/home/themaster/workspace/cse314final/MovieClean/MovieClean/norm.py:33: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  .apply(pd.Series)
/home/themaster/workspace/cse314final/MovieClean/MovieClean/norm.py:33: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  .apply(pd.Series)
/home/themaster/workspace/cse314final/MovieClean/MovieClean/norm.py:33: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  .apply

,company
id,
862,Pixar Animation Studios
8844,TriStar Pictures
15602,Warner Bros.
31357,Twentieth Century Fox Film Corporation
11862,Sandollar Productions
...,...
8985,SHOTS
345775,CPH Film Fund
16,Lantia Cinema & Audiovisivi


In [55]:
countries = norm.json2series(movies[['id','production_countries']].dropna(), 'production_countries', 'name', 'id').rename('country')
countries = norm.id2int(countries)
countries

/home/themaster/workspace/cse314final/MovieClean/MovieClean/norm.py:33: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  .apply(pd.Series)
/home/themaster/workspace/cse314final/MovieClean/MovieClean/norm.py:33: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  .apply(pd.Series)
/home/themaster/workspace/cse314final/MovieClean/MovieClean/norm.py:33: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  .apply(pd.Series)
/home/themaster/workspace/cse314final/MovieClean/MovieClean/norm.py:33: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  .apply

,country
id,
862,United States of America
8844,United States of America
15602,United States of America
31357,United States of America
11862,United States of America
...,...
8985,Slovakia
8985,Slovenia
8985,Spain


In [56]:
clean_movie = movies.drop(columns=['belongs_to_collection', 'genres', 'homepage', 'spoken_languages', 'tagline', 'production_companies', 'production_countries']).set_index('id')
clean_movie = norm.id2int(clean_movie)
clean_movie

,adult,budget,imdb_id,original_language,original_title,overview,popularity,poster_path,release_date,revenue,runtime,status,title,video,vote_average,vote_count
id,,,,,,,,,,,,,,,,
862,False,30000000,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,1995-10-30,373554033.0,81.0,Released,Toy Story,False,7.7,5415.0
8844,False,65000000,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,1995-12-15,262797249.0,104.0,Released,Jumanji,False,6.9,2413.0
15602,False,0,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,1995-12-22,0.0,101.0,Released,Grumpier Old Men,False,6.5,92.0
31357,False,16000000,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,1995-12-22,81452156.0,127.0,Released,Waiting to Exhale,False,6.1,34.0
11862,False,0,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,1995-02-10,76578911.0,106.0,Released,Father of the Bride Part II,False,5.7,173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30840,False,0,tt0102797,en,Robin Hood,"Yet another version of the classic epic, with ...",5.683753,/fQC46NglNiEMZBv5XHoyLuOWoN5.jpg,1991-05-13,0.0,104.0,Released,Robin Hood,False,5.7,26.0
111109,False,0,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,0.178241,/xZkmxsNmYXJbKVsTRLLx3pqGHx7.jpg,2011-11-17,0.0,360.0,Released,Century of Birthing,False,9.0,3.0
67758,False,0,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",0.903007,/d5bX92nDsISNhu3ZT69uHwmfCGw.jpg,2003-08-01,0.0,90.0,Released,Betrayal,False,3.8,6.0


In [59]:
#Checkpoint
clean_movie.to_csv(save_path+'clean_movie.csv', encoding='utf-8')
countries.to_csv(save_path+'countries.csv', encoding='utf-8')
companies.to_csv(save_path+'companies.csv', encoding='utf-8')
languages.to_csv(save_path+'languages.csv', encoding='utf-8')
genres.to_csv(save_path+'genres.csv', encoding='utf-8')
collections.to_csv(save_path+'collections.csv', encoding='utf-8')

# clean ratings

In [7]:
ratings = pd.read_csv(path+"/"+file_names[6])
ratings.head()

,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435
2,1,858,5.0,1425941523
3,1,1221,5.0,1425941546
4,1,1246,5.0,1425941556


In [12]:
ratings = ratings.groupby('movieId')['rating'].mean()

In [16]:
ratings.index.name='id'

In [18]:
ratings.index

Int64Index([     1,      2,      3,      4,      5,      6,      7,      8,
                 9,     10,
            ...
            176253, 176255, 176257, 176259, 176263, 176267, 176269, 176271,
            176273, 176275],
           dtype='int64', name='id', length=45115)

In [19]:
ratings.to_csv(save_path+'ratings.csv', encoding='utf-8')